# Importações

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import zipfile

In [2]:
def load_data_year(year, cols):
    file = f'../../../microdados_anos/MICRODADOS_ENEM_{year}.csv'
    
    try:
        df = pd.read_csv(file, encoding='cp1252', sep=';', usecols=cols, nrows=1000)
    except:
        try:
            df = pd.read_csv(file, encoding='cp1252', usecols=cols, nrows=1000)
        except:
            df = pd.read_csv(file, encoding='cp1252', nrows=1000)
            df = df.rename(columns={'NOTA_LC': 'NU_NOTA_LC', 'NOTA_CN': 'NU_NOTA_CN', 'NOTA_CH': 'NU_NOTA_CH',  
                                    'NOTA_MT': 'NU_NOTA_MT','IN_PRESENCA_MT': 'TP_PRESENCA_MT', 'IDADE': 'NU_IDADE',
                                    'IN_PRESENCA_LC': 'TP_PRESENCA_LC', 'IN_PRESENCA_CH': 'TP_PRESENCA_CH', 
                                    'UF_RESIDENCIA': 'SG_UF_RESIDENCIA',  'ANO_CONCLUIU': 'TP_ANO_CONCLUIU', 
                                     'ST_CONCLUSAO': 'TP_ST_CONCLUSAO', 'IN_PRESENCA_CN': 'TP_PRESENCA_CN'})[cols]
    
    return df

In [3]:
cols_used = ['NU_INSCRICAO', 'NU_ANO', 'NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA',
             'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
             'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
             'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
             'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
             'NU_NOTA_MT', 'NU_NOTA_COMP1',
             'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
             'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
             'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
             'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
             'Q023', 'Q024', 'Q025']

In [4]:
years = [2015, 2016, 2017, 2018, 2019]

In [5]:
def union_datas(years, cols):
    df_final = pd.DataFrame([], columns=cols)
    
    for y in years:
        df_final = pd.concat([df_final, load_data_year(y, cols)])
    
    return df_final

In [6]:
%%time

df_union = union_datas(years, cols_used)

Wall time: 121 ms


In [7]:
df_union.head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,150000000376,2015,Gravataí,RS,42,M,1.0,1,1,10,...,B,A,A,B,A,A,B,B,B,B
1,150000003053,2015,Montes Claros,MG,21,M,0.0,3,1,5,...,A,A,A,B,B,A,D,B,B,B
2,150000001161,2015,Olinda,PE,22,M,1.0,2,4,0,...,A,A,A,A,A,A,C,B,B,A
3,150000001335,2015,Belo Horizonte,MG,23,F,0.0,4,1,6,...,B,A,A,C,A,B,D,B,B,B
4,150000001219,2015,Teófilo Otoni,MG,18,M,0.0,3,1,2,...,A,A,B,B,A,A,C,B,B,B


# Pre-processing

In [8]:
pd.DataFrame({'types': df_union.dtypes, 'missing': df_union.isna().sum()})

,types,missing
NU_INSCRICAO,object,0
NU_ANO,object,0
NO_MUNICIPIO_RESIDENCIA,object,0
SG_UF_RESIDENCIA,object,0
NU_IDADE,object,0
TP_SEXO,object,0
TP_ESTADO_CIVIL,float64,177
TP_COR_RACA,object,0
TP_ST_CONCLUSAO,object,0
TP_ANO_CONCLUIU,object,0


In [9]:
df_final = df_union

In [10]:
#lambda x: pd.Series.mode(x) if (len(pd.Series.mode(x)) <= 1) else pd.Series.mode(x)[0]
def agg_lambda(x):
    mode = pd.Series.mode(x)
   
    if (len(mode) <= 1):
        return mode
    else:
        return mode[0]

aggregations = {
    'TP_ESTADO_CIVIL': agg_lambda,
    'NU_NOTA_CN':'mean',
    'NU_NOTA_CH': 'mean',
    'NU_NOTA_LC': 'mean',
    'NU_NOTA_MT': 'mean',
    'NU_NOTA_COMP1': agg_lambda,
    'NU_NOTA_COMP2': agg_lambda,
    'NU_NOTA_COMP3': agg_lambda,
    'NU_NOTA_COMP4': agg_lambda,
    'NU_NOTA_COMP5': agg_lambda,
    'NU_NOTA_REDACAO': agg_lambda
}
df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).agg(aggregations)
df_groups = df_groups.rename_axis(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).reset_index()

In [11]:
df_groups.head()

,TP_ESCOLA,TP_ST_CONCLUSAO,SG_UF_RESIDENCIA,TP_ESTADO_CIVIL,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
0,1,1,AC,0.0,519.785714,591.357143,545.083333,538.383333,140,140,120.0,120.0,80.0,380.0
1,1,1,AL,0.0,490.832500,550.546341,515.032500,517.546154,120,120,120,120,80,520
2,1,1,AM,0.0,483.160000,553.093939,520.466667,501.696667,120,120,120,120,80,520
3,1,1,AP,0.0,461.900000,506.920000,508.720000,481.976923,120.0,120.0,120.0,120.0,0,0
4,1,1,BA,0.0,497.350198,543.312261,525.167692,499.225000,120,120,120,120,80,580


In [12]:
#imp_ec = SimpleImputer(strategy="most_frequent")
#imp_ec.fit(df_final[['TP_ESTADO_CIVIL']])

In [13]:
#df_final[['TP_ESTADO_CIVIL']] = imp_ec.transform(df_final[['TP_ESTADO_CIVIL']])

In [14]:
#df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).mean()
#df_groups = df_groups.rename_axis(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).reset_index()

In [15]:
#df_groups.head()

In [16]:
def add_null_values(x):
    if(sum(pd.isna(x)) > 0):
        df_x = df_groups.query('TP_ESCOLA==@x.TP_ESCOLA & TP_ST_CONCLUSAO==@x.TP_ST_CONCLUSAO & SG_UF_RESIDENCIA==@x.SG_UF_RESIDENCIA')
    
        for c in df_x.columns:
            if(pd.isna(x[c]) > 0):
                x[c] = df_x[c]
    return x

In [17]:
df_final = df_final.apply(add_null_values, axis=1)
df_final.dropna(inplace=True) # Perguntas do questionario com resposta nula

In [18]:
pd.DataFrame({'types': df_final.dtypes, 'missing': df_final.isna().sum()})

,types,missing
NU_INSCRICAO,int64,0
NU_ANO,int64,0
NO_MUNICIPIO_RESIDENCIA,object,0
SG_UF_RESIDENCIA,object,0
NU_IDADE,int64,0
TP_SEXO,object,0
TP_ESTADO_CIVIL,object,0
TP_COR_RACA,int64,0
TP_ST_CONCLUSAO,int64,0
TP_ANO_CONCLUIU,int64,0


In [19]:
%%time

# Save
#file = '../datasets/integrated_data.csv'
#df_final.to_csv(file, index=False)

#z = zipfile.ZipFile('../datasets/integrated_data.zip', 'w', zipfile.ZIP_DEFLATED)
#z.write(file)    
#z.close()

Wall time: 2.76 s


# Incluindo Status Socioeconômico

In [20]:
df_final.dropna(inplace=True)

In [21]:
df_final.head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,150000000376,2015,Gravataí,RS,42,M,1.0,1,1,10,...,B,A,A,B,A,A,B,B,B,B
1,150000003053,2015,Montes Claros,MG,21,M,0.0,3,1,5,...,A,A,A,B,B,A,D,B,B,B
2,150000001161,2015,Olinda,PE,22,M,1.0,2,4,0,...,A,A,A,A,A,A,C,B,B,A
3,150000001335,2015,Belo Horizonte,MG,23,F,0.0,4,1,6,...,B,A,A,C,A,B,D,B,B,B
4,150000001219,2015,Teófilo Otoni,MG,18,M,0.0,3,1,2,...,A,A,B,B,A,A,C,B,B,B


In [22]:
num = ['Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008',
       'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
       'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024',
       'Q025', 'Q026', 'Q027', 'Q028', 'Q029', 'Q030', 'Q031', 'Q032',
       'Q033', 'Q034', 'Q035', 'Q036', 'Q037', 'Q038', 'Q039', 'Q040',
       'Q041', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047', 'Q048',
       'Q049', 'Q050']

In [23]:
question = ['Até que série seu pai, ou o homem responsável por você, estudou?',
       'Até que série sua mãe, ou a mulher responsável por você, estudou?',
       'A partir da apresentação de algumas ocupações divididas em grupos ordenados, indique o grupo que contempla a ocupação mais próxima da ocupação do seu pai ou do homem responsável por você. (Se ele não estiver trabalhando, escolha uma ocupação pensando no último trabalho dele).',
       'A partir da apresentação de algumas ocupações divididas em grupos ordenados, indique o grupo que contempla a ocupação mais próxima da ocupação da sua mãe ou da mulher responsável por você. (Se ela não estiver trabalhando, escolha uma ocupação pensando no último trabalho dela).',
       'Incluindo você, quantas pessoas moram atualmente em sua residência?',
       'Qual é a renda mensal de sua família? (Some a sua renda com a dos seus familiares.)',
       'Em sua residência trabalha empregado(a) doméstico(a)?',
       'Na sua residência tem banheiro?',
       'Na sua residência tem quartos para dormir?',
       'Na sua residência tem carro?',
       'Na sua residência tem motocicleta?',
       'Na sua residência tem geladeira?',
       'Na sua residência tem freezer (independente ou segunda porta da geladeira)?',
       'Na sua residência tem máquina de lavar roupa? (o tanquinho NÃO deve ser considerado)',
       'Na sua residência tem máquina de secar roupa (independente ou em conjunto com a máquina de lavar roupa)?',
       'Na sua residência tem forno micro-ondas?',
       'Na sua residência tem máquina de lavar louça?',
       'Na sua residência tem aspirador de pó?',
       'Na sua residência tem televisão em cores?',
       'Na sua residência tem aparelho de DVD?',
       'Na sua residência tem TV por assinatura?',
       'Na sua residência tem telefone celular?',
       'Na sua residência tem telefone fixo?',
       'Na sua residência tem computador?',
       'Na sua residência tem acesso à Internet?']

In [24]:
df = pd.DataFrame(list(zip(num,question)), columns = ['num','question'])

In [25]:
df

,num,question
0,Q001,"Até que série seu pai, ou o homem responsável ..."
1,Q002,"Até que série sua mãe, ou a mulher responsável..."
2,Q003,A partir da apresentação de algumas ocupações ...
3,Q004,A partir da apresentação de algumas ocupações ...
4,Q005,"Incluindo você, quantas pessoas moram atualmen..."
5,Q006,Qual é a renda mensal de sua família? (Some a ...
6,Q007,Em sua residência trabalha empregado(a) domést...
7,Q008,Na sua residência tem banheiro?
8,Q009,Na sua residência tem quartos para dormir?
9,Q010,Na sua residência tem carro?


In [34]:
for col in df['num'].values:
    print(col, len(df_final[col].unique()))
    print(df_final[col].unique())

Q001 8
['A' 'E' 'B' 'D' 'F' 'H' 'C' 'G']
Q002 8
['A' 'E' 'C' 'F' 'B' 'D' 'G' 'H']
Q003 6
['A' 'C' 'D' 'F' 'B' 'E']
Q004 6
['A' 'B' 'F' 'E' 'D' 'C']
Q005 14
[ 3.  5.  4.  7.  2.  6.  1. 10.  9.  8. 11. 20. 15. 13.]
Q006 17
['D' 'B' 'F' 'G' 'J' 'E' 'K' 'C' 'P' 'L' 'N' 'O' 'I' 'H' 'A' 'Q' 'M']
Q007 4
['A' 'C' 'B' 'D']
Q008 5
['B' 'C' 'D' 'E' 'A']
Q009 5
['B' 'D' 'C' 'E' 'A']
Q010 5
['A' 'B' 'C' 'D' 'E']
Q011 4
['A' 'B' 'C' 'D']
Q012 5
['B' 'C' 'D' 'A' 'E']
Q013 5
['A' 'B' 'D' 'C' 'E']
Q014 3
['B' 'A' 'C']
Q015 3
['A' 'B' 'C']
Q016 3
['B' 'A' 'C']
Q017 3
['A' 'B' 'C']
Q018 2
['A' 'B']
Q019 5
['B' 'A' 'C' 'D' 'E']
Q020 2
['A' 'B']
Q021 2
['A' 'B']
Q022 5
['B' 'D' 'C' 'E' 'A']
Q023 2
['B' 'A']
Q024 5
['B' 'C' 'D' 'A' 'E']
Q025 2
['B' 'A']


In [35]:
5/8

0.625

# Testes anos

In [27]:
def load_data_year_2015(year, cols):
    file = f'../../../microdados_anos/MICRODADOS_ENEM_{year}.csv'
    try:
        df = pd.read_csv(file, encoding='cp1252', sep=';', usecols=cols, nrows=1000)
    except:
        df = pd.read_csv(file, encoding='cp1252', usecols=cols, nrows=1000)
    
    return df

In [28]:
df_2014 = pd.read_csv('../../../microdados_anos/MICRODADOS_ENEM_2014.csv', encoding='cp1252', nrows=1000)
df_2014.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'COD_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'COD_UF_RESIDENCIA', 'UF_RESIDENCIA',
       'IN_ESTUDA_CLASSE_HOSPITALAR', 'COD_ESCOLA', 'COD_MUNICIPIO_ESC',
       'NO_MUNICIPIO_ESC', 'COD_UF_ESC', 'UF_ESC',
       'ID_DEPENDENCIA_ADM_ESC', 'ID_LOCALIZACAO_ESC', 'SIT_FUNC_ESC',
       'IDADE', 'TP_SEXO', 'NACIONALIDADE', 'COD_MUNICIPIO_NASCIMENTO',
       'NO_MUNICIPIO_NASCIMENTO', 'COD_UF_NASCIMENTO', 'UF_NASCIMENTO',
       'ST_CONCLUSAO', 'ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TP_ENSINO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA',
       'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA', 'IN_SURDO_CEGUEIRA',
       'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL',
       'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_LACTANTE',
       'IN_IDOSO', 'IN_AUTISMO', 'IN_SABATISTA', 'IN_BRAILLE',
       'IN_AMPLIADA_24', 'IN_AMPLIADA_18', 'IN_LEDOR', 'IN_ACESSO',
       'IN_TRANSCRICAO', 'IN_LIBRAS', 'IN_LEITURA_LABIAL

In [29]:
len(['NU_NOTA_LC', 'NU_NOTA_CN', 'NU_NOTA_CH', 'TP_PRESENCA_MT', 'NU_NOTA_MT', 'SG_UF_RESIDENCIA', 'TP_PRESENCA_CH', 
 'NU_IDADE', 'TP_ANO_CONCLUIU', 'TP_PRESENCA_LC', 'TP_ST_CONCLUSAO', 'TP_PRESENCA_CN'])

12

In [30]:
df_2014 = df_2014.rename(columns={'NOTA_LC': 'NU_NOTA_LC', 'NOTA_CN': 'NU_NOTA_CN', 'NOTA_CH': 'NU_NOTA_CH',  'NOTA_MT': 'NU_NOTA_MT',
                        'IN_PRESENCA_MT': 'TP_PRESENCA_MT', 'IN_PRESENCA_CH': 'TP_PRESENCA_CH', 
                        'UF_RESIDENCIA': 'SG_UF_RESIDENCIA', 'IDADE': 'NU_IDADE', 'ANO_CONCLUIU': 'TP_ANO_CONCLUIU', 
                        'IN_PRESENCA_LC': 'TP_PRESENCA_LC', 'ST_CONCLUSAO': 'TP_ST_CONCLUSAO', 'IN_PRESENCA_CN': 'TP_PRESENCA_CN'})

In [31]:
df_2014[cols_used]

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,140000000001,2014,SAO BERNARDO DO CAMPO,SP,18,F,0,1,1,2013.0,...,A,A,A,A,D,B,A,3,4,4.0
1,140000000002,2014,CONTAGEM,MG,24,M,0,2,4,NaN,...,C,D,D,D,D,A,A,5,4,NaN
2,140000000003,2014,SAO PAULO,SP,19,M,0,2,1,2012.0,...,C,A,A,A,D,A,A,5,3,NaN
3,140000000004,2014,SAO PAULO,SP,18,F,0,1,1,2013.0,...,B,D,D,D,D,A,A,4,4,NaN
4,140000000005,2014,VITORIA DA CONQUISTA,BA,45,M,1,1,1,2004.0,...,B,A,A,D,D,B,A,5,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,140000000996,2014,BRASILIA,DF,26,F,0,3,1,2010.0,...,C,A,A,D,D,A,B,5,3,NaN
996,140000000997,2014,FRANCA,SP,26,M,0,1,1,2007.0,...,C,A,D,D,D,B,A,5,5,NaN
997,140000000998,2014,JATAI,GO,19,M,0,3,1,2013.0,...,B,A,D,D,D,A,C,5,5,NaN
998,140000000999,2014,RIO DE JANEIRO,RJ,17,F,0,3,2,NaN,...,C,A,A,D,D,B,C,5,5,NaN


In [32]:
df_2015 = load_data_year(2015, cols_used)
df_2015.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'NO_MUNICIPIO_RESIDENCIA',
       'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001',
       'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009',
       'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017',
       'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025'],
      dtype=object)